In [ ]:
#  dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# reading in beer ratings csv

df=pd.read_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\data\beer_rating.csv')
df.head()


In [ ]:
#  read in usbreweries csv
brdf = pd.read_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\data\usbreweries.csv')
brdf.head()

In [ ]:
# creating brewery df based on the beer ratings csv to create a universaL index for all breweries
df = df[['brewery']].drop_duplicates()
ratings_brew_df = pd.DataFrame(df.groupby('brewery'))
ratings_brewery_df = pd.DataFrame()
ratings_brewery_df['brewery_name'] = ratings_brew_df[0]
ratings_brewery_df.head()


In [ ]:
# df based on open brewery db, to create a universal index for all breweries
bdf = pd.read_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\data\usbreweries.csv')
bdf = bdf[['name']].drop_duplicates()
bdf = pd.DataFrame(bdf.groupby('name'))

grouped_bdf = pd.DataFrame()

grouped_bdf['brewery_name'] = bdf[0]

# grouped_bdf['brewery_name'] = grouped_bdf['brewery_name'].str.replace('(', '')
# grouped_bdf['brewery_name'] = grouped_bdf['brewery_name'].str.replace(')', '')

grouped_bdf.reset_index()

In [ ]:
# merge based on brewery name

merged_df = pd.merge(ratings_brewery_df, grouped_bdf, on=['brewery_name'],how='outer')
merged_df = merged_df.drop_duplicates('brewery_name').reset_index().rename(columns={'index':'brewery_id'})
merged_df.head()


In [ ]:
# create beer : brewery df
rdf = pd.read_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\data\beer_rating.csv')
rdf = rdf[['id','beer name','brewery']]
rdf[['beer_id','beer_name','brewery_name']] = rdf[['id','beer name','brewery']]

beer_to_brewery_df = pd.merge(rdf,merged_df,on=['brewery_name'])
beer_to_brewery_df['brewery'] = merged_df['brewery_id']
beer_to_brewery_df = beer_to_brewery_df.drop(columns=['brewery'])
beer_to_brewery_df = beer_to_brewery_df[['brewery_name','brewery_id','beer_name','beer_id']]
beer_to_brewery_df.head(50)

id_df = beer_to_brewery_df[['brewery_id','beer_id']]
id_df.head()

In [ ]:
# create brewery info df
bdf = pd.read_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\data\usbreweries.csv')
bdf = bdf.drop(columns=['Unnamed: 0','phone','country'])

brewery_info_df = bdf.rename(columns={'name':'brewery_name'})
brewery_info_df.head()

brewery_info_df = pd.merge(brewery_info_df, merged_df, on=['brewery_name'],how='outer')

brewery_info_df.reindex(columns=['brewery_id','brewery_name','brewery_type','city','state','country','latitude','longitude'])
brewery_info_df.sort_values(by=['brewery_id'])

brewery_info_df.head()

In [ ]:
# create beer profile df
profile_df = pd.read_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\data\beer_rating.csv')
profile_df = profile_df[['id','beer name','beer type','ABV','Min IBU','Max IBU','Ave Rating','Description']]
profile_df['avg_ibu'] = (profile_df['Min IBU'] + profile_df['Max IBU'])/2
profile_df = profile_df.drop(columns=['Min IBU','Max IBU'])
profile_df = profile_df.rename(columns={'id':'beer_id','beer name':'beer_name','beer type':'beer_type','ABV':'abv','Ave Rating':'avg_rating','Description':'desc'})
profile_df = profile_df[['beer_id','beer_name','beer_type','abv','avg_rating','avg_ibu','desc']]
profile_df.head()

In [ ]:
# create beer flavor profile df
flavor_df = pd.read_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\data\beer_rating.csv')
flavor_df = flavor_df[['id','beer name','Astringency','Body','Alcohol','Bitter','Sweet','Sour','Salty','Fruits','Hoppy','Spices','Malty']]
flavor_df = flavor_df.rename(columns={'id':'beer_id','beer name':'beer_name','Astringency':'astringency','Body':'body',
                                    'Alcohol':'alcohol','Bitter':'bitter','Sweet':'sweet','Sour':'sour','Salty':'salty',
                                    'Fruits':'fruits','Hoppy':'hoppy','Spices':'spices','Malty':'malty'})
                                

In [ ]:
id_df.to_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\dataframes\beer_id.csv',index=False)
brewery_info_df.to_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\dataframes\brewery_info.csv',index=False)
profile_df.to_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\dataframes\beer_profile.csv',index=False)
flavor_df.to_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\dataframes\beer_flavor.csv',index=False)

In [ ]:
# sql alchemy to create a database
from sqlalchemy import create_engine


In [ ]:
from config import postgres_password
engine = create_engine('postgresql://postgres:'+ postgres_password +'@localhost:5432/mybeer')

id_df.to_sql('beer_id',engine,if_exists='append')
brewery_info_df.to_sql('brewery_info',engine,if_exists='append')
profile_df.to_sql('beer_profile',engine,if_exists='append')
flavor_df.to_sql('beer_flavor',engine,if_exists='append')

engine.dispose()

In [ ]:
id_df.to_json(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\json\beer_id.json',orient='records')
brewery_info_df.to_json(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\json\brewery_info.json',orient='records')
profile_df.to_json(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\json\beer_profile.json',orient='records')
flavor_df.to_json(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\json\beer_flavor.json',orient='records')

In [62]:
# df for beer rankings
top_beers = pd.read_csv(r"C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\data\beer_rating.csv")
# top_beers.head()
top_beers['Avg IBU'] = (top_beers['Min IBU'] + top_beers['Max IBU'])/2
top_beers = top_beers.drop(columns=['Name','Min IBU','Max IBU','id'])

top_beers = top_beers.rename(columns={'brewery':'Brewery', 'beer name':'Beer Name', 'beer type':'Beer Type'})
top_beers = top_beers[['Beer Name','Brewery','Beer Type','ABV','Avg IBU','Ave Rating','Astringency','Body','Alcohol','Bitter','Sweet','Sour','Salty','Fruits','Hoppy','Spices','Malty']]
# top_beers = pd.merge(top_beers,beer_to_brewery_df,on=['beer_id'])
# top_beers[['beer_id','beer_name','brewery_name','beer_type','abv','avg_rating','avg_ibu','desc','astringency','body','alcohol','bitter','sweet','sour','salty','fruits','hoppy','spices','malty']] = top_beers[['beer_id','beer_name_x','brewery_name','beer_type','abv','avg_rating','avg_ibu','desc','astringency','body','alcohol','bitter','sweet','sour','salty','fruits','hoppy','spices','malty']]
top_beers.to_json(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\json\top_beers.json',orient='records')
# # top_beers.to_csv(r'dataframes/top_beers.csv',index=False)
top_beers.head()


,Beer Name,Brewery,Beer Type,ABV,Avg IBU,Ave Rating,Astringency,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty
0,Amber,Alaskan Brewing Co.,Altbier,5.3,37.5,3.65,13,32,9,47,74,33,0,33,57,8,111
1,Double Bag,Long Trail Brewing Co.,Altbier,7.2,37.5,3.90,12,57,18,33,55,16,0,24,35,12,84
2,Long Trail Ale,Long Trail Brewing Co.,Altbier,5.0,37.5,3.58,14,37,6,42,43,11,0,10,54,4,62
3,Doppelsticke,Uerige Obergärige Hausbrauerei,Altbier,8.5,37.5,4.15,13,55,31,47,101,18,1,49,40,16,119
4,Scurry,Off Color Brewing,Altbier,5.3,37.5,3.67,21,69,10,63,120,14,0,19,36,15,218


In [ ]:
placement = [1,2,3,4,5,6,7,8,9,10]

top_ABV = top_beers.sort_values(by=['ABV'],ascending=False).head(10)
top_ABV['Placement'] = placement

top_ABV =  top_ABV[ ['Placement'] + [col for col in top_ABV.columns if col != 'Placement']] \
                                    .set_index(['Placement'])
top_ABV = top_ABV[['Beer Name','ABV','Brewery','Beer Type']]
top_ABV.to_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\flavor_csv\top_ABV.csv')
top_ABV.head(10)

for col in top_ABV.columns:
    print(top_ABV[col].values)


In [ ]:
top_IBU = top_beers.sort_values(by=['Avg IBU'],ascending=False).head(10)
top_IBU['Placement'] = placement
top_IBU =  top_IBU[ ['Placement'] + [col for col in top_IBU.columns if col != 'Placement']].set_index(['Placement'])
top_IBU = top_IBU[['Beer Name','Avg IBU','Brewery','Beer Type']]
top_IBU.to_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\flavor_csv\top_IBU.csv')
top_IBU.head(10)

for col in top_IBU.columns:
    print(top_IBU[col].values)


In [ ]:
top_rating = top_beers.sort_values(by=['Ave Rating'],ascending=False).head(10)
top_rating['Placement'] = placement
top_rating =  top_rating[ ['Placement'] + [col for col in top_rating.columns if col != 'Placement']].set_index(['Placement'])
top_rating = top_rating[['Beer Name','Ave Rating','Brewery','Beer Type']]
top_rating.to_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\flavor_csv\top_rtg.csv')
top_rating.head(10)

In [ ]:
top_astringency = top_beers.sort_values(by=['Astringency'],ascending=False).head(10)
top_astringency['Placement'] = placement
top_astringency =  top_astringency[ ['Placement'] + [col for col in top_astringency.columns if col != 'Placement']].set_index(['Placement'])
top_astringency = top_astringency[['Beer Name','Astringency','Brewery','Beer Type']]
top_astringency.to_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\flavor_csv\top_ast.csv')
top_astringency.head(10)

for col in top_astringency.columns:
    print(top_astringency[col].values)

In [ ]:
top_body = top_beers.sort_values(by=['Body'],ascending=False).head(10)
top_body['Placement'] = placement
top_body =  top_body[ ['Placement'] + [col for col in top_body.columns if col != 'Placement']].set_index(['Placement'])
top_body = top_body[['Beer Name','Body','Brewery','Beer Type']]
top_body.to_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\flavor_csv\top_bdy.csv')
top_body.head(10)

for col in top_body.columns:
    print(top_body[col].values)

In [ ]:
top_alcohol = top_beers.sort_values(by=['Alcohol'],ascending=False).head(10)
top_alcohol['Placement'] = placement
top_alcohol =  top_alcohol[ ['Placement'] + [col for col in top_alcohol.columns if col != 'Placement']].set_index(['Placement'])
top_alcohol = top_alcohol[['Beer Name','Alcohol','Brewery','Beer Type']]
top_alcohol.to_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\flavor_csv\top_alc.csv')
top_alcohol.head(10)

for col in top_alcohol.columns:
    print(top_alcohol[col].values)


In [ ]:
top_bitter = top_beers.sort_values(by=['Bitter'],ascending=False).head(10)
top_bitter['Placement'] = placement
top_bitter =  top_bitter[ ['Placement'] + [col for col in top_bitter.columns if col != 'Placement']].set_index(['Placement'])
top_bitter = top_bitter[['Beer Name','Bitter','Brewery','Beer Type']]
top_bitter.rename(columns={'Bitter':'Bitterness'},inplace=True)
top_bitter.to_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\flavor_csv\top_btr.csv')
top_bitter.head(10)

for col in top_bitter.columns:
    print(top_bitter[col].values)


In [ ]:
top_sweet = top_beers.sort_values(by=['Sweet'],ascending=False).head(10)
top_sweet['Placement'] = placement
top_sweet =  top_sweet[ ['Placement'] + [col for col in top_sweet.columns if col != 'Placement']].set_index(['Placement'])
top_sweet = top_sweet[['Beer Name','Sweet','Brewery','Beer Type']]
top_sweet.rename(columns={'Sweet':'Sweetness'},inplace=True)
top_sweet.to_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\flavor_csv\top_swt.csv')
top_sweet.head(10)

for col in top_sweet.columns:
    print(top_sweet[col].values)
    

In [ ]:
top_sour = top_beers.sort_values(by=['Sour'],ascending=False).head(10)
top_sour['Placement'] = placement
top_sour =  top_sour[ ['Placement'] + [col for col in top_sour.columns if col != 'Placement']].set_index(['Placement'])
top_sour = top_sour[['Beer Name','Sour','Brewery','Beer Type']]
top_sour.rename(columns={'Sour':'Sourness'},inplace=True)
top_sour.to_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\flavor_csv\top_sou.csv')
top_sour.head(10)

for col in top_sour.columns:
    print(top_sour[col].values)

In [ ]:
top_salty = top_beers.sort_values(by=['Salty'],ascending=False).head(10)
top_salty['Placement'] = placement
top_salty =  top_salty[ ['Placement'] + [col for col in top_salty.columns if col != 'Placement']].set_index(['Placement'])
top_salty = top_salty[['Beer Name','Salty','Brewery','Beer Type']]
top_salty.rename(columns={'Salty':'Saltiness'},inplace=True)
top_salty.to_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\flavor_csv\top_slt.csv')
top_salty.head(10)

for col in top_salty.columns:
    print(top_salty[col].values)


In [ ]:
top_fruits = top_beers.sort_values(by=['Fruits'],ascending=False).head(10)
top_fruits['Placement'] = placement
top_fruits =  top_fruits[ ['Placement'] + [col for col in top_fruits.columns if col != 'Placement']].set_index(['Placement'])
top_fruits = top_fruits[['Beer Name','Fruits','Brewery','Beer Type']]
top_fruits.rename(columns={'Fruits':'Fruitiness'},inplace=True)
top_fruits.to_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\flavor_csv\top_frt.csv')
top_fruits.head(10)

for col in top_fruits.columns:
    print(top_fruits[col].values)

In [ ]:
top_hoppy = top_beers.sort_values(by=['Hoppy'],ascending=False).head(10)
top_hoppy['Placement'] = placement
top_hoppy =  top_hoppy[ ['Placement'] + [col for col in top_hoppy.columns if col != 'Placement']].set_index(['Placement'])
top_hoppy = top_hoppy[['Beer Name','Hoppy','Brewery','Beer Type']]
top_hoppy.rename(columns={'Hoppy':'Hoppiness'},inplace=True)
top_hoppy.to_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\flavor_csv\top_hop.csv')
top_hoppy.head(10)

for col in top_hoppy.columns:
    print(top_hoppy[col].values)
    

In [ ]:
top_spices = top_beers.sort_values(by=['Spices'],ascending=False).head(10)
top_spices['Placement'] = placement
top_spices =  top_spices[ ['Placement'] + [col for col in top_spices.columns if col != 'Placement']].set_index(['Placement'])
top_spices = top_spices[['Beer Name','Spices','Brewery','Beer Type']]
top_spices.to_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\flavor_csv\top_spc.csv')
top_spices.head(10)

for col in top_spices.columns:
    print(top_spices[col].values)
    


In [ ]:
top_malty = top_beers.sort_values(by=['Malty'],ascending=False).head(10)
top_malty['Placement'] = placement
top_malty =  top_malty[ ['Placement'] + [col for col in top_malty.columns if col != 'Placement']].set_index(['Placement'])
top_malty = top_malty[['Beer Name','Malty','Brewery','Beer Type']]
top_malty.rename(columns={'Malty':'Maltiness'},inplace=True)
top_malty.to_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\flavor_csv\top_mlt.csv')
top_malty.head(10)

for col in top_malty.columns:
    print(top_malty[col].values)

In [ ]:
top_rating = top_beers.sort_values(by=['Ave Rating'],ascending=False).head(10)
top_rating['Placement'] = placement
top_rating =  top_rating[ ['Placement'] + [col for col in top_rating.columns if col != 'Placement']].set_index(['Placement'])
top_rating = top_rating[['Beer Name','Ave Rating','Brewery','Beer Type']]
top_rating.to_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\flavor_csv\top_rat.csv')
top_rating.head(10)

for col in top_rating.columns:
    print(top_rating[col].values)

In [63]:
top_fruits = top_beers.sort_values(by=['Fruits'],ascending=False).head(10)
top_fruits['Placement'] = placement
top_fruits =  top_fruits[ ['Placement'] + [col for col in top_fruits.columns if col != 'Placement']].set_index(['Placement'])
top_fruits = top_fruits[['Beer Name','Fruits','Brewery','Beer Type']]
top_fruits.rename(columns={'Fruits':'Fruitiness'},inplace=True)
top_fruits.to_csv(r'C:\Users\hmwhi\OneDrive\Desktop\GitHub\Project3-Team1\flavor_csv\top_frt.csv')
top_fruits.head(10)

for col in top_fruits.columns:
    print(top_fruits[col].values)

['Sour In The Rye - Kumquats' 'King Julius' 'Juice Machine' 'Very Green'
 'JJJuliusss' 'Femme Fatale Yuzu Pale' 'Montmorency Vs Balaton'
 'Serendipity' 'Doubleganger' 'Society & Solitude #4']
[222 217 211 209 207 207 204 202 201 200]
['Bruery Terreux' 'Tree House Brewing Company'
 'Tree House Brewing Company' 'Tree House Brewing Company'
 'Tree House Brewing Company' 'Evil Twin Brewing' 'Jester King Brewery'
 'New Glarus Brewing Company' 'Tree House Brewing Company'
 'Hill Farmstead Brewery']
['Wild Ale' 'IPA - New England' 'IPA - New England' 'IPA - New England'
 'IPA - New England' 'Brett Beer' 'Wild Ale' 'Fruit and Field Beer'
 'IPA - New England' 'IPA - New England']
